In [ ]:
# 선물, 현물 데이터 불러오기
import pandas as pd
FData = pd.read_csv("../dataset/kospi200_F202312_45min.csv", index_col=0)
PData = pd.read_csv("../dataset/kospi200_30min.csv", index_col=0)

In [ ]:
# 선물 데이터 확인 및 전처리 계획
    # 음수 -> 양수로
    # drop
    # 컬럼명 변경
FData.head(5)

In [ ]:
FData.info()

In [ ]:
FData = FData.abs()
FData

In [ ]:
# FData.drop('Unnamed: 0',axis = 1,inplace=True)

In [ ]:
FData = FData.loc[:,['체결시간','현재가']]

In [ ]:
FData.columns = ['체결시간','선물_시가']

In [ ]:
FData.columns

In [ ]:
FData.head()

In [ ]:
# 체결시간 15:30분 데이터만 남김
# str로 바꿔줘서 나중에 datetime으로 바꿔줘야함
FData_df = FData[FData['체결시간'].astype(str).str.contains('153000')]
FData_df

In [ ]:
# 체결시간 뒤에 날리고 날짜로 맞춤
FData_df['체결시간'] = FData_df['체결시간'].astype(str).str[:8]
FData_df

In [ ]:
# '선물_시가' 오른차순 정렬
FData_df_sorted = FData_df.sort_values(by='체결시간', ascending=True)

# 인덱싱 다시해주기
FData_df_sorted = FData_df_sorted.reset_index(drop=True)

FData_df_sorted

In [ ]:
# 현물 데이터 확인 및 전처리 계획
    # 음수 -> 양수
    # 100으로 나눠주기
    # drop
PData.head(5)

In [ ]:
PData = PData.abs()

In [ ]:
cols_to_divide = ['현재가','시가','고가','저가']
PData[cols_to_divide] = PData[cols_to_divide] / 100

In [ ]:
# PData.drop('Unnamed: 0',axis = 1,inplace=True)

In [ ]:
PData = PData.loc[:,['체결시간','시가']]
PData

In [ ]:
PData.columns = ['체결시간','현물_시가']

In [ ]:
PData.columns

In [ ]:
PData.head()

In [ ]:
# 체결시간 15:30분 데이터만 남김
# str로 바꿔줘서 나중에 datetime으로 바꿔줘야함
PData_df = PData[PData['체결시간'].astype(str).str.contains('153000')]
PData_df

In [ ]:
# 체결시간 뒤에 날리고 날짜로 맞춤
PData_df['체결시간'] = PData_df['체결시간'].astype(str).str[:8]
PData_df

In [ ]:
# '선물_시가' 오른차순 정렬
PData_df_sorted = PData_df.sort_values(by='체결시간', ascending=True)

# 인덱싱 다시해주기
PData_df_sorted = PData_df_sorted.reset_index(drop=True)

PData_df_sorted

In [ ]:
# 데이터 합치기
Data = pd.merge(FData_df_sorted,PData_df_sorted,how='inner',on='체결시간')

In [ ]:
# 데이터셋 완성
Data

### 베이시스=선물 가격−현물 가격
* 양수인 경우 (콘탱고, Contango):
    - 의미: 선물 가격이 현물 가격보다 높은 상태를 가리킵니다.
    - 원인: 여러 요인이 있을 수 있으나, 주로 보관 비용, 금리, 보험료 등의 요소가 포함될 수 있습니다. 예를 들어, 선물 계약의 만기까지 원자재를 저장하는 데 발생하는 비용이나 금융 비용이 선물 가격에 반영되어 현물 가격보다 높아질 수 있습니다.
    - 특징: 콘탱고 상태에서는 선물 계약의 만기가 가까워질수록 베이시스는 일반적으로 감소하며, 만기일에는 선물 가격과 현물 가격이 수렴하게 됩니다.
* 음수인 경우 (백워데이션, Backwardation):
    - 의미: 선물 가격이 현물 가격보다 낮은 상태를 가리킵니다.
    - 원인: 수급 불균형, 특정 상품에 대한 급격한 수요 증가, 저장 공간의 부족 등 다양한 요인이 원인이 될 수 있습니다. 백워데이션은 특정 상품에 대한 현재의 실제 수요가 미래의 예상 수요보다 높을 때 발생하기도 합니다.
    - 특징: 백워데이션 상태에서는 선물 계약의 만기가 가까워질수록 선물 가격과 현물 가격은 수렴하게 됩니다.

### 베이시스 값 해석

* 베이시스가 양수 (포지티브 베이시스)
     - 이는 트레이더들이 미래의 가격이 현재 가격보다 높을 것으로 예상한다는 것을 의미합니다. 
     - 이러한 상황에서는 보통 "캐리 트레이드" 전략이 유리할 수 있습니다.

* 베이시스가 음수 (네거티브 베이시스)
    - 이는 트레이더들이 미래의 가격이 현재 가격보다 낮을 것으로 예상한다는 것을 의미합니다. 
    - 이러한 상황에서는 선물 계약을 롤오버하는 비용이 발생할 수 있습니다.

In [ ]:
# basis 계산
Data['basis'] = Data['선물_']-Data['현물_시가']

In [ ]:
Data.head()

In [ ]:
Data.info()

In [ ]:
# basis_data = Data[['체결시간','basis_시가']]

In [ ]:
# Data.to_csv('basis_2403.csv', index=False)

In [ ]:
# basis_data.head()

### 베이시스 괴리율

* (베이시스 / 현물 가격) * 100 

In [ ]:
# basis 계산
Data['basis_disparate_ratio'] = ((Data['basis'] / Data['현물_시가']) * 100).round(3)
Data

### 베이시스 리스크

* 현물 가격 변동 - 선물 가격 변동
    1. 현물 가격 변동 구하기
        - 새로운 컬럼 '현물_가격변동' 만들기
        - (i+1)번째 행의 값 - i번쨰 행의 값 구해서 '현물_가격변동' 컬럼의 i+1번째 행에 대입
    2. 선물 가격 변동은 현물 가격 변동 구하는 방법과 동일
         - 새로운 컬럼 명은 '선물_가격변동'
    3. 현물가격변동 - 선물가격변동 값 구해서 새로운 컬럼 'basis_risk'에 대입


##### 베이시스 리스크 값 의미
* 양수 값
    - 현물 가격의 변동이 선물 가격의 변동보다 큰 경우를 나타냅니다. 
    - 이는 선물 계약을 통해 리스크를 완전히 헤지할 수 없다는 것을 의미합니다.
    - 선물 가격의 변동이 현물 가격의 변동을 따라가지 못하는 경우에 베이시스 리스크가 발생합니다.

* 음수 값
    - 현물 가격의 변동이 선물 가격의 변동보다 작은 경우를 나타냅니다.
    - 이는 선물 가격의 변동이 현물 가격의 변동을 초과하여 움직였다는 것을 의미합니다.

 = > 베이시스 리스크는 헤지 전략을 사용하는 투자자나 기업에게 중요한 개념입니다. 선물 계약을 사용하여 리스크를 완전히 제거하려는 의도가 있더라도 베이시스 리스크로 인해 완전한 헤지가 어려울 수 있습니다. 따라서 투자자나 기업은 베이시스 리스크를 고려하여 헤지 전략을 조절해야 합니다.
        


In [ ]:
# 선물, 현물의 정제 데이터 불러오기.
Data.head()

In [ ]:
Data.info()

In [ ]:
# 1. 현물 가격 변동 구하기
Data['현물_가격변동'] = Data['현물_시가'].diff().abs()

# 2. 선물 가격 변동 구하기
Data['선물_가격변동'] = Data['선물_시가'].diff().abs()

Data.head()


In [ ]:
Data['basis_risk'] = Data['현물_가격변동']-Data['선물_가격변동']
Data.head()

In [ ]:
Data.info()

In [ ]:
# basis, basis_risk만 남기기~!!

Data = Data[['체결시간','basis','basis_risk','basis_disparate_ratio']]
Data.rename(columns={'체결시간': 'date'}, inplace=True)
Data['date'] = pd.to_datetime(Data['date'])

In [ ]:
Data.dtypes

In [ ]:
Data.fillna(method='bfill', inplace=True)

In [ ]:
Data.isna().sum()

In [ ]:
Data = Data.round(3)

print(Data)

In [ ]:
Data = Data.iloc[:-1, :]
print(Data)

In [ ]:
Data.to_csv('basis.csv', index=False)